<a href="https://colab.research.google.com/github/sepide2005/repository1/blob/main/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Network Class Definition

In [ ]:
import numpy as np
from scipy.special import expit
import matplotlib.pyplot as plt

class NeuralNetwork :

  #initialize the neural network
  def __init__(self, input, hidden, output, learning_rate):
    #set the number of nodes in input , hidden and output layars
    self.input_nodes = input
    self.hidden_nodes = hidden
    self.output_nodes = output

    #set the number of learning_rate
    self.learning_rate = learning_rate

    #Apply the sigmoid function
    self.activation_function = lambda x : expit(x)

    #pow(self.input_nodes,-0.5) = 1\((Number of connections)^(1/2))
    self.w_i_h = np.random.default_rng().normal(0, pow(self.input_nodes,-0.5), (self.hidden_nodes, self.input_nodes))
    #pow(self.hidden_nodes,-0.5) = 1\((Number of connections)^(1/2))
    self.w_h_o = np.random.default_rng().normal(0, pow(self.hidden_nodes,-0.5), (self.output_nodes, self.hidden_nodes))
    pass


  # Calculation of the output by giving the input - Feedforward
  def query(self, input_list) :
    inputs = np.array(input_list, ndmin=2).T

    x_hidden = np.dot(self.w_i_h, inputs)
    o_hidden = self.activation_function(x_hidden)

    x_output = np.dot(self.w_h_o, o_hidden)
    o_output = self.activation_function(x_output)

    return o_output


  #Learning networks based on examples - Backpropagation
  def train(self, input_list, target_list) :

    inputs = np.array(input_list, ndmin=2).T

    x_hidden = np.dot(self.w_i_h, inputs)
    o_hidden = self.activation_function(x_hidden)

    x_output = np.dot(self.w_h_o, o_hidden)
    o_output = self.activation_function(x_output)


    targets = np.array(target_list, ndmin=2).T
    output_error = targets - o_output
    hidden_error = np.dot(self.w_h_o.T, output_error)

    #Calculation of new weights of connections with Gradient descent
    self.w_h_o += self.learning_rate * np.dot((output_error * o_output * (1 - o_output)), o_hidden.T)
    self.w_i_h += self.learning_rate * np.dot((hidden_error * o_hidden * (1 - o_hidden)), inputs.T)

# Neural Network Instance Creation

In [ ]:
#calling the train data from MNIST dataset
from google.colab import drive
drive.mount('/content/drive')
train_file = open('drive/MyDrive/mnist/mnist_train.csv','r')
train_list = list(train_file)
train_file.close()

#calling the train data from MNIST dataset
test_file = open('drive/MyDrive/mnist/mnist_test.csv','r')
test_list = list(test_file)
test_file.close()

ValueError: mount failed

In [ ]:
#determining the number of each neuron in each layar
input_nodes = 784
hidden_nodes = 100
output_nodes = 10

#determining the learning rate of the network
learning_rate = 0.1

epochs = 4

#example
nn = NeuralNetwork(input=input_nodes, hidden=hidden_nodes, output=output_nodes, learning_rate=learning_rate)
print("initial weights (w_input_hidden):",nn.w_i_h)

#training the created network with MNIST training data
for e in range(epochs):
  for row in train_list:
    row_data = row.split(',')
    inputs = (np.asfarray(row_data[1:]) / (255.0 * 0.99)) + 0.01
    targets = np.zeros(output_nodes) + 0.01
    targets[int(row_data[0])] = 0.99
    nn.train(inputs, targets)
print("weights after training (w_input_hidden):",nn.w_i_h)


In [ ]:
#testing the created network with one of the datas of testing data
test_row_data = test_list[1].split(',')
print(test_row_data)
print('target test is : ', test_row_data[0], '')

#displaying the MNIST dataset
image_data = np.asfarray(test_row_data[1:]).reshape((28,28))
image = plt.imshow(image_data, cmap='Greys')

#network prediction
output = nn.query((np.asfarray(test_row_data[1:]) / (255.0 * 0.99)) + 0.01)
print("Predicted digit is :", np.argmax(output))

In [ ]:
#testing the performance of the network in terms of predicting all test data
score_card = []
for row in test_list:
  row_data = row.split(',')
  target_number = int(row_data[0])
  inputs = (np.asfarray(row_data[1:]) / (255.0 * 0.99)) + 0.01
  outputs = nn.query(inputs)
  Predicted_number = np.argmax(outputs)
  # print('\ntarget number :', target_number)
  # print('predicted_number :', Predicted_number)
  if target_number == Predicted_number :
    score_card.append(1)
  else :
    score_card.append(0)

#Network Performance Report
# print('\nscore : ', score_card)
score_card_array = np.asarray(score_card)
print('\nperformance : ', '%', (score_card_array.sum()/score_card_array.size)*100)
